In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import resnet18, ResNet50_Weights
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
import pandas as pd


In [34]:
image_dir = "Images/"
train_txt = "train_test_files/5_folders_cross_validations_files/cross_validation_1/train_1.txt"
test_txt = "train_test_files/5_folders_cross_validations_files/cross_validation_1/test_1.txt"


In [35]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2
])

In [36]:
def load_data(txt_file, image_dir):
    data = pd.read_csv(txt_file, delim_whitespace=True, header=None)
    image_paths = [image_dir + img_name for img_name in data[0]]
    scores = data[1].values
    return image_paths, scores

def create_dataloader(image_paths, scores, batch_size=64):
    images = []
    for img_path in image_paths:
        image = Image.open(img_path).convert('RGB')
        image = transform(image)
        images.append(image)

    images = torch.stack(images)
    scores = torch.tensor(scores, dtype=torch.float32)

    dataset = TensorDataset(images, scores)


    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

In [37]:
train_image_paths, train_scores = load_data(train_txt, image_dir)
test_image_paths, test_scores = load_data(test_txt, image_dir)

train_loader = create_dataloader(train_image_paths, train_scores)
test_loader = create_dataloader(test_image_paths, test_scores)

/var/folders/_h/lfj08q4n53s2rb5h08cln0640000gn/T/ipykernel_28114/3863767359.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(txt_file, delim_whitespace=True, header=None)
/var/folders/_h/lfj08q4n53s2rb5h08cln0640000gn/T/ipykernel_28114/3863767359.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(txt_file, delim_whitespace=True, header=None)


In [38]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=False)
# model = torch.nn.DataParallel(model)

# # Set the model to run on the device
# model = model.to(device)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [39]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [40]:
num_epochs = 10  
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    progress = 0
    for inputs, targets in train_loader:
        progress += 1
        print(f'Progress: {progress}/{len(train_loader)}', end='\r')
        optimizer.zero_grad()
        # inputs = inputs.to(device)
        # targets = targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    

model.eval()
running_loss = 0.0

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        running_loss += loss.item() * inputs.size(0)

test_loss = running_loss / len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')



KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'resnet18_finetuned.pth')
